### Two types of functions
- buit-in function - round,avg,sum,max,min
- User Defined function - allow custom logic when built-in SQL/Spark functions are not enough.

### Two types
 - Temporary function
 - Permanent function

In [0]:

df = spark.table("inceptez_catalog.inputdb.tbltxns")
df = spark.read.format("csv").option("inferSchema",True).load("/Volumes/inceptez_catalog/inputdb/customerdata/txns")
df = spark.sql("select * from inceptez_catalog.inputdb.tbltxns")
display(df)

In [0]:
def calculate_discount(amount):
    if amount > 100:
        disamt = amount * 0.1
    elif amount > 50:
        disamt = amount * 0.05
    else:
        disamt = 0.0
    return disamt

def calculate_sellingprice(amt,dis):
    return amt - dis

In [0]:
from pyspark.sql.functions import udf, round, col
from pyspark.sql.types import DoubleType

discount_udf = udf(calculate_discount, DoubleType())

df1 = df.withColumn("discountamt",discount_udf(col("amount")))
df1.display()


In [0]:
spark.udf.register("discount", calculate_discount)
df.createOrReplaceTempView("tbltxns")
df1 = spark.sql("select *, discount(amount) as discountamt from tbltxns")


In [0]:
spark.udf.register("sellprice", calculate_sellingprice)
df1.createOrReplaceTempView("tbltxns")
spark.sql("select *, sellprice(amount,discountamt) as sellprice from tbltxns").display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION inceptez_catalog.inputdb.calculate_discount(amount DOUBLE)
RETURNS DOUBLE
RETURN
    CASE 
        WHEN amount > 50 THEN amount * 0.05     -- 5% discount
        WHEN amount >= 100 THEN amount * 0.1    -- 10% discount
        ELSE 0.0                                  -- no discount
    END;

In [0]:
%sql

select amount,round(inceptez_catalog.inputdb.calculate_discount(amount),2) as sellprice from inceptez_catalog.inputdb.tbltxns